# Fake News Classifier Using LSTM

In [1]:
#Dataset: https://www.kaggle.com/c/fake-news/data#

In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('fake-news/train.csv')

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
###Drop Nan Values
df=df.dropna()

In [5]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [6]:
## Get the Dependent features
y=df['label']

In [7]:
X.shape

(18285, 4)

In [8]:
y.shape

(18285,)

In [9]:
import tensorflow as tf

In [10]:
tf.__version__

'2.15.0'

In [11]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [12]:
### Vocabulary size
voc_size=5000

# Onehot Representation

In [13]:
messages=X.copy()

In [14]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [15]:
messages.reset_index(inplace=True)

In [16]:
import nltk
import re
from nltk.corpus import stopwords

In [17]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Ved\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [18]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    #print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [19]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [20]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[164, 1409, 3825, 1480, 3022, 2389, 2125, 518, 1419, 3928],
 [3947, 1879, 983, 1933, 4806, 555, 3056],
 [2073, 4073, 4942, 3046],
 [3239, 2110, 2083, 3784, 2417, 1067],
 [2318, 4806, 4078, 688, 3428, 2001, 4806, 359, 24, 4185],
 [1024,
  2817,
  1274,
  3236,
  904,
  3002,
  3698,
  4902,
  1651,
  4197,
  4577,
  1539,
  3990,
  3919,
  3056],
 [382, 2931, 361, 3501, 3263, 1965, 1755, 3058, 2803, 4424, 4368],
 [2525, 1677, 1035, 2156, 766, 4731, 3002, 110, 2803, 4424, 4368],
 [1892, 2188, 711, 4446, 3783, 317, 4903, 4925, 3002, 4778],
 [1876, 379, 3872, 1914, 643, 2706, 4711, 3488],
 [1852, 4225, 2724, 1807, 2478, 4534, 2331, 311, 1236, 1180, 1746],
 [3784, 2986, 3022, 317, 3002, 766],
 [781, 2863, 1890, 2314, 4528, 1707, 4893, 1933, 1034],
 [1180, 3169, 1305, 1434, 2562, 3288, 3169, 2803, 4424, 4368],
 [3281, 3518, 3136, 1454, 3842, 2803, 4424, 4368],
 [3535, 4108, 487, 2768, 3934, 4280, 2340, 4186, 3632, 936],
 [1647, 3596, 1879],
 [2305, 2116, 4629, 1519, 3002, 1875, 3035, 3056],

# Embedding Representation

In [21]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ...  518 1419 3928]
 [   0    0    0 ... 4806  555 3056]
 [   0    0    0 ... 4073 4942 3046]
 ...
 [   0    0    0 ... 2803 4424 4368]
 [   0    0    0 ...  315  440 4833]
 [   0    0    0 ... 4588 4842 3532]]


In [22]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,  164,
       1409, 3825, 1480, 3022, 2389, 2125,  518, 1419, 3928])

In [23]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256501 (1001.96 KB)
Trainable params: 256501 (1001.96 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [24]:
len(embedded_docs),y.shape

(18285, (18285,))

In [25]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [26]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [27]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

# Model Training

In [28]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10


192/192 [==============================] - 7s 23ms/step - loss: 0.3343 - accuracy: 0.8468 - val_loss: 0.1905 - val_accuracy: 0.9188
Epoch 2/10
192/192 [==============================] - 4s 21ms/step - loss: 0.1381 - accuracy: 0.9460 - val_loss: 0.1901 - val_accuracy: 0.9221
Epoch 3/10
192/192 [==============================] - 4s 20ms/step - loss: 0.0912 - accuracy: 0.9680 - val_loss: 0.2087 - val_accuracy: 0.9211
Epoch 4/10
192/192 [==============================] - 4s 19ms/step - loss: 0.0630 - accuracy: 0.9791 - val_loss: 0.2485 - val_accuracy: 0.9145
Epoch 5/10
192/192 [==============================] - 4s 19ms/step - loss: 0.0378 - accuracy: 0.9882 - val_loss: 0.2691 - val_accuracy: 0.9170
Epoch 6/10
192/192 [==============================] - 4s 18ms/step - loss: 0.0267 - accuracy: 0.9912 - val_loss: 0.3580 - val_accuracy: 0.9162
Epoch 7/10
192/192 [==============================] - 4s 19ms/step - loss: 0.0128 - accuracy: 0.9962 - val_loss: 0.4392 - val_accuracy: 0.91

# Adding Dropout

In [29]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

# Performance Metrics And Accuracy

In [31]:
y_pred=model.predict(X_test)

189/189 [==============================] - 2s 7ms/step


In [38]:
y_pred_ = [1 if i>=0.5 else 0 for i in y_pred]

In [37]:
y_test

array([1, 0, 0, ..., 0, 1, 1], dtype=int64)

In [34]:
from sklearn.metrics import confusion_matrix

In [39]:
confusion_matrix(y_test,y_pred_)

array([[2728,  691],
       [1737,  879]], dtype=int64)

In [41]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred_)

0.5976801988400994